# Deep Reinforcement Learning - Collaboration and Competition





# Overview

The goal of this agent is to train a multi-agent scenario where two agents are interacting with eachother and learning from their experiences. The score measured here at the end of each episode is the score of the agent who has scored higher.

The environment provided is that of a Unity ML Agents scenario.

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1. If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01. Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Each agent receives its own, local observation. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping.


# Implementation

The project was solved using Deep Reinforcement Learning using the Multi-Agent version of DDPG(Deep Deterministic Policy Gradient) Algorithm.

The base code of the project is derived from my implementation of the previous project in Deep Reinforcement Learning Nanodegree for Continuous Control in a Reacher environment where I had used DDPG.

https://github.com/DebankurS/UdacityDRLND-Continuous-Control

The same solution was modified and updated for the new environment.
In the previous environment there were 20 Agents which interacted with the environment independently.
The same code was re-used since in this case, both the agents were trained while they interacted with eachother.

* The notebook **Tennis.ipynb** contains the implementation for training and visualising the untrained agent. Then the training code is implemented.

* **ddpg_agent.py** contains the code to understand and determine how the agent interacts with the environment and learns to optimize the reward.

* **model.py** contains the architecture of the deep learning model used in this implementation.


# Learning Algorithm

The Deep Deterministic Policy Gradient algorithm(DDPG) was chosen to solve the agent-environment interaction, modified for Multi-Agent Scenario.

Here I will be explaining how DDPG algorithm works in general and the changes made for our scenario.

In general, the more commonly known **Q-Learning** method utilizes a state-action return value and from there chooses the actions which maximise the expected reward, hence reaching the optimal policy.

**Policy gradients** on the other hand tries to direcly map the states to the actions where a neural network can be used as a function approximator, by increasing the probabilities of the actions which yield higher returns, leading to the optimal policy.

DDPG combines these two ideas in the form of an actor-critic algorithm. It is based on the deterministic policy gradient that can operate over continuous action spaces.

The Actor network here maps the states to the actions while the Critic network tries to estimate the Q function based on the actions selected by the Actor. The actor on the other hand learns the policy based off the Critic network.

It also utilizes the replay buffer to learn from past experiences in the way DQN Algorithm does.
Furthermore Ornstein-Uhlenbeck process is utilized to generate a noise in order to encourage exploration.

Another implementation borrowed from the DQN Algorithm is the usage of the target and local networks which here has been used for both the actor and critic network.


### Modification from the original DDPG implementation for Multi-Agent Scenario

In our current environment the agent learns by interacting with eachother. In this case I am reusing the same Actor-Critic Network with the mindset that we learn both positive or negative outcomes from mutually opposing agents who have in other words 'learnt' the from eachother's experiences.

### Model Architecture
There are two model architectures used here defined in the file model.py :

#### Actor

* The model has 2 fully connected layers with 400 and 300 nodes respectively.
* It takes in an input equal to the state size provided which in this case is 24
* Relu activation function is used betweeen the two layers
* Batch normalisation has been applied since it has been proven that it speeds up the learning in case of DDPG
* The output node of this layer is of size equal to the action size


#### Critic

* The model has 2 fully connected layers with 400 nodes in the first layer and (300+action_size) in the second.
* It takes in an input equal to the state size provided which in this case is 24
* Relu activation function is used betweeen the two layers
* Batch normalisation has been applied since it has been proven that it speeds up the learning in case of DDPG
* The output node of this layer is of size equal to 1 which means that it outputs the expected return value
* torch.cat operation is applied between the first two layers to denote the mapping between the states and actions as defined in the algorithm

### Agent

The agent is defined in the agent.py file. It is the Deep Q-Learning agent which interacts with the environment. It references the local and target network from the model defined in model.py.

It contains four methods:

**Step**: Here the agent saves the experiences in the replay memory. After a certain set of predefined intervals, it also causes the network to learn from the replay buffer a certain number of times.

**Act**: Here the agent returns the action determined by the local Actor Network. The output is of size 4 corresponding to each actions but in the range of -1 to 1 as expected by the network. Furthermore, noise is added via the Ornstein-Uhlenbeck process to encourage exploration.

**Learn**: This is where the agent actually learns. 

#### Critic network learning
We randomly sample a batch from the experience buffer in the form of (states, actions, rewards, next_states, dones) and pass on the next_states to the actor target network to determine the next set of actions which in return is passed on to the critic target network.

The return from the critic target network is actually the Q values determined by the target network.

Again, like in the DQN Algorithm, we compute the expected Q values from these next states and compute the Mean Squared Error loss between Q_targets and Q_expected and update the critic target network accordingly.

# Actor network learning
Here we get the predicted actions from the local actor network based on the current states.
The loss is computed as the mean of the Q values corresponding to the different state action pairs.
We use the negative sign here because we want to maximise the gradients and hence using gradient ascent.

Then we run the soft_update function to update the target network with the local network parameters

**Soft_update**: Here we update the target networks with the local networks parameters using the formula
                    
                    θ_target = τ*θ_local + (1 - τ)*θ_target

### Hyperparameters

The hyperparameters used to train the agent are:



* BUFFER_SIZE = int(1e6)  # replay buffer size
* BATCH_SIZE = 256        # minibatch size
* GAMMA = 0.99            # discount factor
* TAU = 1e-3              # for soft update of target parameters
* LR_ACTOR = 2e-4         # learning rate of the actor 
* LR_CRITIC = 2e-4        # learning rate of the critic
* WEIGHT_DECAY = 0        # L2 weight decay
* UPDATE_EVERY=1         # Update interval
* NUM_UPDATES=1          # Number of learning steps after every predefined interval in UPDATE_EVERY

# Plot of Rewards

The plot of average rewards for every 100 episodes are listed below.

* Episode 100	Average Score: 0.00	Score: 0.00
* Episode 200	Average Score: 0.01	Score: 0.00
* Episode 300	Average Score: 0.00	Score: 0.00
* Episode 400	Average Score: 0.02	Score: 0.09
* Episode 500	Average Score: 0.09	Score: 0.10
* Episode 600	Average Score: 0.11	Score: 0.20
* Episode 700	Average Score: 0.42	Score: 0.70
* Episode 722	Average Score: 0.50	Score: 1.10
* Environment solved in 722 episodes!	Average Score: 0.50

![](Multi_Agent.png)

## Ideas for Future Work

The Reinforcement Learning agent was trained using Deep Deterministic Policy Gradients extended to work over multiple agents which are interacting over multiple agents

* Here in this case I have used Random Sampling as in the previous project, but Prioritized Experience Replay might show better results in comparison.

* I might need to optimize the code further to improve the stability since the agent does start to learn quite later in the journey and as shown in the graph, gives huge spikes in final scores.

* The agent currently converges after variable number of steps even with the same hyperparameters. Further modification to them might yield more consistent results.

* I stopped training when the target goal was reached, but further training for a longer period of time might lead to more insights on how long the current agent maintains its stability